In [14]:
import keras
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import tensorflow as tf
from keras.models import Model
import ipynb.fs.full.Structural_Perturbations as SP
import ipynb.fs.full.MultiTest as MT
tf.logging.set_verbosity(tf.logging.ERROR)
import sys
from keras.layers import Input, Dense, Lambda, merge,add,concatenate
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.models import load_model

In [15]:
train_data,train_labels,eval_data,eval_labels = SP.load(data='fmnist')

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


In [16]:
train_labels = keras.utils.to_categorical(train_labels, 10)
eval_labels = keras.utils.to_categorical(eval_labels, 10)

In [17]:
class MNIST:
    def __init__(self):
        self.build_model()
        
    def build_model(self):
        self.inputs = Input(shape=(784, ),name='imgs')
        self.labels = Input((10,),name='labels')
        inputs = keras.layers.core.Reshape((28,28,1))(self.inputs)
        labels = self.labels

        conv1 = Conv2D(8, kernel_size=(3, 3),
                         activation='relu',
                         )(inputs)

        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(8, (3, 3), activation='relu')(pool1)

        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        pool2_flat = Flatten()(pool2)

        dense1 = Dense(64, activation='relu')(pool2_flat)
        dropout = Dropout(0.5)(dense1)

        logits = Dense(10)(dropout)

        self.predictions = Activation('softmax')(logits)

        g = merge([logits, pool2, labels], mode=lambda x: K.gradients(x[0] * x[2], x[1]), output_shape=(pool2.shape))
        cost = merge([labels, self.predictions], mode=lambda x: (-1) * K.sum(x[0] * K.log(x[1]), axis=1), output_shape=labels.shape)
        gb_grad = merge([cost, inputs], mode=lambda x: K.gradients(x[0], x[1]), output_shape=(inputs.shape))

In [18]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

m = MNIST()

model = Model(inputs=[m.inputs, m.labels], outputs=[m.predictions])

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

/home/uttaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/uttaran/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/uttaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/uttaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [5]:
model = load_model('./keras/models/mnist_test1.h5')

In [21]:
model.fit([train_data,train_labels],train_labels,
          batch_size=256,
          epochs=50,
          verbose=1)
model.save('./keras/models/fmnist_smaller.h5')
model.summary()

Epoch 1/50
55000/55000 [==============================] - 2s 39us/step - loss: 0.4142 - acc: 0.8496
Epoch 2/50
55000/55000 [==============================] - 2s 38us/step - loss: 0.4112 - acc: 0.8516
Epoch 3/50
55000/55000 [==============================] - 2s 37us/step - loss: 0.4058 - acc: 0.8528
Epoch 4/50
55000/55000 [==============================] - 2s 38us/step - loss: 0.4044 - acc: 0.8548
Epoch 5/50
55000/55000 [==============================] - 2s 38us/step - loss: 0.3999 - acc: 0.8545
Epoch 6/50
55000/55000 [==============================] - 2s 39us/step - loss: 0.3973 - acc: 0.8555
Epoch 7/50
55000/55000 [==============================] - 2s 39us/step - loss: 0.3930 - acc: 0.8569
Epoch 8/50
55000/55000 [==============================] - 2s 40us/step - loss: 0.3908 - acc: 0.8589
Epoch 9/50
55000/55000 [==============================] - 2s 40us/step - loss: 0.3874 - acc: 0.8598
Epoch 10/50
55000/55000 [==============================] - 2s 39us/step - loss: 0.3885 - acc: 0.8598

In [22]:
model.evaluate([eval_data,eval_labels],eval_labels)

10000/10000 [==============================] - 1s 81us/step


[0.30761797652244566, 0.8895]